<a href="https://colab.research.google.com/github/Shahid0120/pytorch-mini-projects/blob/main/chp_6_deep_dive_into_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Chapter 6
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt

# Section 1 - Layer and Modules

## Exercise 1

In [ ]:
# Chp 6.1 Exercise 1
# What kinds of problems will occur if you change MySequential to
#store modules in a Python list?
# Current implementation
class MySequential(nn.Module):
  def __init__(self, *args):
      super().__init__()
      for idx, module in enumerate(args):
          self.add_module(str(idx), module)

  def forward(self, X):
    for module in self.children():
        X = module(X)
    return X


In [ ]:
# Sequential with list implementation
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    self.module_arr = []
    for idx, module in enumerate(args):
      """
      Example input:
        MySequential(
          nn.Linear()
          F.relu()
          nn.Linear()
        )
      """
      self.module_arr.append(module)


  def forward(self, X: torch.Tensor) -> torch.Tensor :
    for module in enumerate(self.module_arr):
        # apply transform sequentially
        X = module(X)
    return

problem : in the constructure we never initialiszed the hiddne layer transformation, thus python is having a problem applky the transformation since it was object was never initialized!

## Exercise 2

Q. Implement a module that takes two modules as an argument, say net1 and net2 and returns the concatenated output of both networks in the forward propagation. This is also called a parallel module.

In [19]:
# Random class net1
class Net1(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=1, out_features=1)
    self.relu = nn.ReLU()
    self.layer_2 = nn.Linear(in_features=1, out_features=1)

  def forward(self, X:torch.Tensor) -> torch.Tensor:
    return self.layer_2(self.relu(self.layer_1(X)))

# Random class net2
class Net2(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=1, out_features=1)
    self.leaky_relu = nn.LeakyReLU()
    self.layer_3 = nn.Linear(in_features=1, out_features=1)

  def forward(self, X:torch.Tensor) -> torch.Tensor:
    return self.layer_3(self.leaky_relu(self.layer_1(X)))

# Parallel Module
class ParallelNet1Net2(Net1, Net2):
  def __init__(self):
    super().__init__()
    self.net1 = Net1()
    self.net2 = Net2()

  def concate(self, x:torch.Tensor) -> torch.Tensor:
    net1_output = self.net1.forward(x)
    net2_ouput = self.net2.forward(x)
    return net1_output + net2_ouput

x = torch.randn(1)

# Net1 initialisation
net1 = Net1()
net1_x = net1.forward(x).item()

# Net2 initialisation
net2 = Net2()
net2_x = net1.forward(x).item()

# Parallel initialisation
parallel = ParallelNet1Net2()
parallel_x = parallel.concate(x).item()


print(f"X : {x.item()} | Net_1 : {net1_x} | Net_2 : {net2_x} | Parallel : {parallel_x}")




X : 0.7477195262908936 | Net_1 : -1.2419242858886719 | Net_2 : -1.2419242858886719 | Parallel : 0.38656148314476013


## Random Recall from Chapter 6

In [23]:
# Using nn.Sequential
import torch.nn.functional as F

class RandomNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(in_features=1, out_features=10),
        nn.ReLU(),
        nn.Linear(in_features=10, out_features=10),
        nn.ReLU(),
        nn.Linear(in_features=10, out_features=1)
    )

  def forward(self, X:torch.Tensor) -> torch.Tensor:
    return self.layers(X)

x = torch.rand(1)
random_network = RandomNetwork()
random_network_x = random_network.forward(x)

print(f"X : {x.item()} | Random Network : {random_network_x.item()}")


X : 0.6129076480865479 | Random Network : -0.4373367428779602


# Section 2 - Paramter Management

In [46]:
class RandomNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(in_features=1, out_features=10),
        nn.ReLU(),
        nn.Linear(in_features=10, out_features=10),
        nn.ReLU(),
        nn.Linear(in_features=10, out_features=1)
    )

  def forward(self, X:torch.Tensor) -> torch.Tensor:
    return self.layers(X)

X = torch.rand(1)
random_network = RandomNetwork()
random_network_x = random_network.forward(X)

print(f"X : {x.item()} | Random Network : {random_network_x.item()}")
print(f"Paramter of Random Network : {random_network.state_dict()}")
print(f"Bias Paramter for layer 1 {random_network.layers[0].bias.data}")

for index,layer in enumerate(random_network.layers):
  try:
    print(f"Index : {index} | Layer : {layer} \n {random_network.layers[index].bias.data}")
  except:
    continue


X : 0.6129076480865479 | Random Network : 0.3765067756175995
Paramter of Random Network : OrderedDict([('layers.0.weight', tensor([[ 0.0495],
        [-0.2563],
        [ 0.2484],
        [ 0.3130],
        [-0.0187],
        [-0.9413],
        [-0.1802],
        [ 0.9323],
        [ 0.5465],
        [-0.3764]])), ('layers.0.bias', tensor([ 0.7994, -0.2375, -0.9554, -0.1615,  0.6156, -0.6021,  0.5208,  0.0344,
        -0.2199, -0.0063])), ('layers.2.weight', tensor([[-0.0623,  0.0986, -0.0783,  0.1477,  0.1602, -0.0996,  0.1328,  0.1704,
          0.0602,  0.1955],
        [ 0.2854,  0.1868,  0.3144,  0.0576,  0.1373,  0.0299,  0.1351, -0.0996,
         -0.2343,  0.0757],
        [-0.2132,  0.1305,  0.1689,  0.2491, -0.2253,  0.0010,  0.0974,  0.0106,
          0.2375,  0.1194],
        [ 0.1727,  0.2207,  0.1660, -0.0159,  0.1599, -0.2343, -0.1310,  0.1077,
         -0.2478,  0.2949],
        [ 0.1477, -0.1895, -0.0634, -0.0521,  0.0963, -0.0011, -0.1975,  0.0981,
         -0.2022, -0

# Section 5 - Custom Layers

In [51]:
# Custom Layers - No paramters
class MeanLayer(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, X:torch.Tensor) -> torch.Tensor :
    return X - X.mean()

net = nn.Sequential(
    nn.LazyLinear(out_features=25),
    MeanLayer(),
    nn.LazyLinear(out_features=1)
)

X = torch.rand(1, 9)

print(f"X : {X} | Net : {net(X).item()}")


X : tensor([[0.6308, 0.6679, 0.0726, 0.4424, 0.5295, 0.7657, 0.4185, 0.4542, 0.8662]]) | Net : -0.2034977674484253


In [70]:
# Custom Layer - With paramters
from torch.nn.parameter import Parameter

class LinearFc(nn.Module):
  def __init__(self, in_units, units):
    super().__init__()
    self.weight = nn.Parameter(data=torch.rand(in_units, units))
    self.bias = nn.Parameter(data=torch.rand(units,))

  def forward(self, X:torch.Tensor) -> torch.Tensor :
    return torch.matmul(self.weight, X) + self.bias

network = nn.Sequential(
    nn.LazyLinear(10),
    LinearFc(10, 1),
    nn.ReLU()
)

X = torch.rand(1, 10)

print(f"X:\n  \n{X}\n  \nNetwork:\n \n{network(X)}\n")





X:
  
tensor([[0.6675, 0.4793, 0.1532, 0.6025, 0.8826, 0.1669, 0.3418, 0.4608, 0.7979,
         0.3946]])
  
Network:
 
tensor([[0.6735, 0.6340, 0.5587, 0.6629, 0.5646, 0.6866, 0.6228, 0.5700, 0.6023,
         0.5785],
        [0.6843, 0.6336, 0.5370, 0.6707, 0.5446, 0.7012, 0.6193, 0.5516, 0.5930,
         0.5625],
        [0.8281, 0.6288, 0.2490, 0.7745, 0.2787, 0.8943, 0.5723, 0.3062, 0.4689,
         0.3491],
        [0.7313, 0.6321, 0.4430, 0.7046, 0.4577, 0.7642, 0.6039, 0.4715, 0.5524,
         0.4928],
        [0.6393, 0.6351, 0.6271, 0.6382, 0.6278, 0.6407, 0.6340, 0.6283, 0.6318,
         0.6292],
        [0.7076, 0.6329, 0.4905, 0.6874, 0.5016, 0.7323, 0.6117, 0.5120, 0.5729,
         0.5280],
        [0.7584, 0.6312, 0.3886, 0.7242, 0.4076, 0.8006, 0.5951, 0.4251, 0.5290,
         0.4526],
        [0.6743, 0.6340, 0.5571, 0.6635, 0.5631, 0.6877, 0.6225, 0.5687, 0.6016,
         0.5774],
        [0.8321, 0.6287, 0.2410, 0.7773, 0.2713, 0.8996, 0.5710, 0.2994, 0.4654,
       